## 计算子网间的地理距离和网络距离绘制线性关系图
* 同级别联赛，按国家划分子网，先取所有国家*2015赛季*、*第一级联赛*数据(转会前后均为第一级联赛)
* 可视化
* 子网距离计算

In [1]:
import numpy as np
import pandas as pd

### 从_clubs__league.csv_提取含有第一级联赛的记录

In [20]:
df_clubs=pd.read_csv('clubs_league.csv')
df_1lv_2015=df_clubs[(df_clubs['season_id']==2015)&(df_clubs['league_level']=='First tier \xc2\xa0-')]
print 'total records: ',df_1lv_2015.shape[0]
print 'total countries: ',len(df_1lv_2015['country_id'].unique())
print 'total countries(country name): ',len(df_1lv_2015['country_name'].unique())
print 'total clubs: ',len(df_1lv_2015['club_id'].unique())
print 'total clubs(club name): ',len(df_1lv_2015['club_name'].unique())
print 'total leagues: ',len(df_1lv_2015['league_id'].unique())
print 'total leagues(league name): ',len(df_1lv_2015['league_name'].unique())

total records:  1343
total countries:  81
total countries(country name):  81
total clubs:  1161
total clubs(club name):  1160
total leagues:  98
total leagues(league name):  92


由以上的基础统计信息可以看到
* total clubs < total records: 可能含有同一个俱乐部对应多条记录，需要看看这些重复的俱乐部是什么情况
* total leagues > total countries: 很明显同一个国家的一级联赛可能不止一个，导出这些国家，看看具体是什么原因

In [3]:
# league_group=df_1lv_2015.groupby([df_1lv_2015['country_id'],df_1lv_2015['league_id']]).size()
league_group=df_1lv_2015.groupby([df_1lv_2015['country_name'],df_1lv_2015['country_id'],df_1lv_2015['league_id']]).size().reset_index(name='counts')
# df_league_group=pd.DataFrame(df_1lv_2015.groupby([df_1lv_2015['league_id'],df_1lv_2015['country_id']]).size().rename('cnts'))
country_group=league_group.groupby([league_group['country_name'],league_group['country_id']]).size().reset_index(name='cnts')
# country_group=pd.DataFrame(league_group.groupby([league_group['country_id'],league_group['league_id']]).size().rename('cnts'))
lst_dup_country=country_group['country_name'][country_group['cnts']>1].values.tolist()
# print league_group
# print country_group
print lst_dup_country
# get the records of those countries with multiple first tier league
# df_dup_country=df_1lv_2015[df_1lv_2015['country_id'].isin(lst_dup_country)]
# print df_dup_country

['Chile', 'Colombia', 'Costa Rica', 'Ecuador', 'India', 'Japan', 'Kazakhstan', 'Mexico', 'Peru', 'Romania', 'Uruguay', 'Venezuela']


In [57]:
def getleague_name(country_name):
    df_theCountry=df_1lv_2015[df_1lv_2015['country_name']==country_name]
    league_theCountry=df_theCountry.groupby([df_1lv_2015['league_name'],df_1lv_2015['league_id']]).size()
    return league_theCountry
print getleague_name('Kazakhstan')

league_name                        league_id
premier-league-relegation-playoff  KAZA          6
premier-liga                       KAS1         12
dtype: int64


先逐个分析上述出现同一级别多个联赛的国家是什么原因：_注：以2015赛季为准_
* 分夏、冬季两场锦标赛，俱乐部是同样的俱乐部，但联赛名不同：(Clausura:winter/Apertura:summer)
 * 美洲：['Chile',,'Uruguay','Ecuador','Colombia','Venezuela']
 * 亚洲：['Japan']
* 分多场锦标赛，但是有一两场参赛球队数目不同或者是展开类似championship,relegation/playoff之类的比赛
 * 美洲： ['Costa Rica'(4)，'Mexico'(3),'Peru'(4),]
 * 欧洲：['Kazakhstan','Romania']
* 两个独立联赛
 * 亚洲： ['India']  i-league更官方，indian-super-league则稍逊一点，根据[quara](https://www.quora.com/What-is-the-difference-between-the-Indian-Super-League-and-the-I-League)的信息

再来看看重复的_club__id_是什么原因

In [31]:
club_group=df_1lv_2015.groupby([df_1lv_2015['club_name'],df_1lv_2015['club_id']]).size().reset_index(name='cnts')
# print club_group
lst_dup_club=club_group['club_id'][club_group['cnts']>1].values.tolist()
print 'No. of duplicate clubs: ',len(lst_dup_club)
df_dup_club=df_1lv_2015[df_1lv_2015['club_id'].isin(lst_dup_club)]
print 'No. of duplicate records: ',df_dup_club.shape[0]
df_dup_club=df_dup_club.sort_values('club_id')
# print df_dup_club
# df_dup_club.to_csv('dup_club_1lv_2015.csv',index=False)

No. of duplicate clubs:  162
No. of duplicate records:  344


原来是存在162个俱乐部属于不同联赛，造成这一现象的原因应该是和之前分析的同一个国家的同一级联赛出现的不同季节性/阶段性的锦标赛有关，比如

* Venezuela
```
club_name,league_id,club_id,country_id,season_id,league_level,league_name,country_name
Caracas FC,VZ1C,531,182,2015,First tier  -,torneo-clausura,Venezuela
Caracas FC,VZ1A,531,182,2015,First tier  -,torneo-apertura,Venezuela
```
* Japan
```
club_name,league_id,club_id,country_id,season_id,league_level,league_name,country_name
Gamba Osaka,JL11,596,77,2015,First tier  -,meiji-yasuda-j1-league-first-stage,Japan
Gamba Osaka,JL12,596,77,2015,First tier  -,meiji-yasuda-j1-league-second-stage,Japan
```
不妨看一下具体是哪些国家，是否与之前的那些国家相对应

In [16]:
lst_countries=list(set(df_dup_club['country_name'].values.tolist()))
print lst_countries

['Romania', 'Peru', 'Uruguay', 'Kazakhstan', 'Mexico', 'Costa Rica', 'Chile', 'Japan', 'Colombia', 'Venezuela', 'Ecuador']


恰好是这些有特殊情况的国家的一级联赛俱乐部，造成‘重复记录’的原因也是因为其league_id,league_name不同

顺便看下有些club,league重名是什么情况

In [33]:
league_id_group=df_1lv_2015.groupby([df_1lv_2015['league_name'],df_1lv_2015['league_id']]).size().reset_index(name='counts')
# print league_id_group.sort_values('league_name')
league_name_group=league_id_group.groupby(league_id_group['league_name']).size().reset_index(name='cnts')
# print league_name_group[league_name_group['cnts']>1]
lst_dup_league_name=league_name_group['league_name'][league_name_group['cnts']>1].values.tolist()
# print lst_dup_league_name
df_dup_league_name=df_1lv_2015[df_1lv_2015['league_name'].isin(lst_dup_league_name)]
print df_dup_league_name.groupby([df_dup_league_name['league_name'],df_dup_league_name['league_id'],df_dup_league_name['country_name']]).size()

league_name                league_id  country_name
premier-league             GB1        England         20
                           GHPL       Ghana           16
                           MAL1       Malta           12
premier-liga               KAS1       Kazakhstan      12
                           RU1        Russia          16
                           UKR1       Ukraine         14
primera-division-apertura  CHLA       Chile           16
                           URU1       Uruguay         16
primera-division-clausura  CHL1       Chile           16
                           URUC       Uruguay         16
dtype: int64


In [36]:
club_id_group=df_1lv_2015.groupby([df_1lv_2015['club_name'],df_1lv_2015['club_id']]).size().reset_index(name='counts')
club_name_group=club_id_group.groupby(club_id_group['club_name']).size().reset_index(name='cnts')
lst_dup_club_name=club_name_group['club_name'][club_name_group['cnts']>1].values.tolist()
df_dup_club_name=df_1lv_2015[df_1lv_2015['club_name'].isin(lst_dup_club_name)]
print df_dup_club_name.groupby([df_dup_club_name['club_name'],df_dup_club_name['club_id'],df_dup_club_name['country_name']]).size()

club_name                club_id  country_name
CD Universidad Católica  3277     Chile           2
                         17584    Ecuador         2
dtype: int64


由此可见确实存在联赛名或者俱乐部名称完全相同的情形，不过可以用*id*加以区分

league_name                          league_id
meiji-yasuda-j1-league-first-stage   JL11         18
meiji-yasuda-j1-league-second-stage  JL12         18
dtype: int64
